In [1]:
# 导入必要的库
import os
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import torch.fft
import random

In [ ]:
# 检查CUDA是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Data_path = '/home/ubuntu/data/workspace/Li/Data/'
save_path = '/home/ubuntu/data/workspace/Li/Model/Python_code_MSE/'
random.seed(42)

In [3]:
M = 9900
batch_size = 32
criterion = nn.MSELoss().to(device)

# 定义数据集类
class SignalDataset(Dataset):
    def __init__(self, generation_signals, labels, experimental_signal):
        self.generation_signals = torch.tensor(generation_signals, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
        self.experimental_signal = torch.tensor(experimental_signal, dtype=torch.float32)

    def __len__(self):
        return len(self.generation_signals)

    def __getitem__(self, idx):
        return {
            'generation_signal': self.generation_signals[idx],
            'labels': self.labels[idx],
            'experimental_signal': self.experimental_signal[idx]
        }

In [4]:
class ConvAutoencoder(nn.Module):
    def __init__(self, signal_length, label_dim, output_shape, hidden_dim, dropout_rate):
        super(ConvAutoencoder, self).__init__()
        
        # 输入信号处理路径（编码器部分）
        self.encoder_signal = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=hidden_dim//4, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=hidden_dim//4, out_channels=hidden_dim//8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=hidden_dim//8, out_channels=hidden_dim//16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        # 标签处理路径（MLP）
        self.label_fc = nn.Sequential(
            nn.Linear(label_dim, hidden_dim//4),
            nn.ReLU(),
            nn.Linear(hidden_dim//4, hidden_dim//8),
            nn.ReLU(),
            nn.Linear(hidden_dim//8, hidden_dim//16)
        )
        
        # 合并后的输出路径（解码器部分）
        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=hidden_dim//16 + hidden_dim//16, out_channels=hidden_dim//8, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=hidden_dim//8, out_channels=hidden_dim//4, kernel_size=2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose1d(in_channels=hidden_dim//4, out_channels=1, kernel_size=6, stride=2)
        )
        
        # 最后的输出层
        self.output_layer = nn.Linear(signal_length, output_shape)

    def forward(self, signal, label):
        # 处理信号 [batch, 1, signal_length]
        signal_features = self.encoder_signal(signal)  # 编码后的特征 [batch, hidden_dim//16, signal_length // 8]
        
        # 处理标签 [batch, label_dim]
        label_features = self.label_fc(label)  # [batch, hidden_dim//64]
        
        # 将标签特征扩展为与信号特征相同的长度
        label_features = label_features.unsqueeze(2).repeat(1, 1, signal_features.size(2))
        
        # 合并特征 [batch, hidden_dim//16 + hidden_dim//64, signal_length // 8]
        combined = torch.cat([signal_features, label_features], dim=1)

        # 解码 [batch, 1, signal_length]
        decoded = self.decoder(combined)
        
        # 输出 [batch, output_shape]
        output = self.output_layer(decoded.view(decoded.size(0), -1))
        return output.view(-1, 1, M)  # 保持输入输出形状一致

In [5]:
def load_data(generation_signal_path, experimental_signal_path, M):
    generation_signals = []  # 存储生成信号
    labels = []  # 存储标签 (Force_label, HI_label, Distance_label, Time_label)
    experimental_signals = []  # 存储输出参数
    filenames = []  # 存储文件名

    # 加载生成信号数据
    generation_dataPaths = sorted(glob.glob(os.path.join(generation_signal_path, '**', '*.csv'), recursive=True))
    for dataPath in generation_dataPaths:

        filename = os.path.basename(dataPath)
        parts = filename.split('_')
        Force_label = float(parts[0])
        HI_label = float(parts[1])
        Distance_label = float(parts[3])
        Time_label = float(parts[2])

        data_1 = pd.read_csv(dataPath)
        for N in range(len(data_1)):
            if data_1.iloc[N, 1] != 0:
                break
        if N < len(data_1):
            generation_signal = data_1.iloc[N:N+M, 1].values
        else:
            continue

        # 分别存储 generation_signal 和 labels
        generation_signals.append(generation_signal)
        labels.append([Force_label, HI_label, Distance_label, Time_label])
        filenames.append(filename)

    # 加载实验信号数据
    experimental_dataPaths = sorted(glob.glob(os.path.join(experimental_signal_path, '**', '*.csv'), recursive=True))
    for dataPath in experimental_dataPaths:

        data_1 = pd.read_csv(dataPath)
        filename = os.path.basename(dataPath)

        for N in range(len(data_1)):
            if data_1.iloc[N, 1] != 0:
                break
        if N < len(data_1):
            experimental_signal = data_1.iloc[N:N+M, 1].values
            experimental_signals.append(experimental_signal)  # 将 experimental_signal 添加到对应的 outputs 元素中

    # 确保 generation_signals、labels 和 outputs 的长度一致
    if len(generation_signals) != len(experimental_signals) or len(labels) != len(experimental_signals):
        print("Error: The number of inputs and outputs does not match.")
        return None, None, None, None, None

    # 转换为 NumPy 数组
    generation_signals = np.array(generation_signals, dtype=float)
    labels = np.array(labels, dtype=float)
    experimental_signals = np.array(experimental_signals, dtype=float)

    return generation_signals, labels, experimental_signals, filenames

In [ ]:
generation_signal_path = f'{Data_path}Hanning_signal_generation'
experimental_signal_path = f'{Data_path}Hanning_Signal_Experiment(10000)'

# 加载数据
filenames = os.listdir(generation_signal_path)  # 假设两个文件夹的文件名是一一对应的

# 控制加载的数据量
n = 100  # 假设只加载 n% 的数据
num_samples = int(len(filenames) * n / 100)
sampled_filenames = random.sample(filenames, num_samples)

# 根据采样的文件名加载数据
# 假设 load_data 函数已经定义好
generation_signals, labels, experimental_signal, filenames = load_data(generation_signal_path, experimental_signal_path, M)

# 将所有数据打乱
shuffled_indices = np.arange(len(generation_signals))
np.random.shuffle(shuffled_indices)

generation_signals = generation_signals[shuffled_indices]
labels = labels[shuffled_indices]
experimental_signal = experimental_signal[shuffled_indices]

# 随机划分训练集、验证集和测试集
# 比例为 4:2:4，即 40% 训练集，20% 验证集，40% 测试集
train_val_indices, test_indices = train_test_split(range(len(sampled_filenames)), test_size=0.4)
train_indices, val_indices = train_test_split(train_val_indices, test_size=0.166)  # 20% / (40% + 20%) = 1/3

# 提取测试集的文件名
test_filenames = [sampled_filenames[i] for i in test_indices]

# 其余代码保持不变
generation_signals_train = generation_signals[train_indices]
labels_train = labels[train_indices]
experimental_signal_train = experimental_signal[train_indices]

generation_signals_val = generation_signals[val_indices]
labels_val = labels[val_indices]
experimental_signal_val = experimental_signal[val_indices]

generation_signals_test = generation_signals[test_indices]
labels_test = labels[test_indices]
experimental_signal_test = experimental_signal[test_indices]

# 增加一个维度，从 [batch_size, N] 变为 [batch_size, 1, N]
generation_signals_train = np.expand_dims(generation_signals_train, axis=1)
experimental_signal_train = np.expand_dims(experimental_signal_train, axis=1)

generation_signals_val = np.expand_dims(generation_signals_val, axis=1)
experimental_signal_val = np.expand_dims(experimental_signal_val, axis=1)

generation_signals_test = np.expand_dims(generation_signals_test, axis=1)
experimental_signal_test = np.expand_dims(experimental_signal_test, axis=1)

print(f'generation_signals_train:{generation_signals_train.shape}')
print(f'generation_signals_val:{generation_signals_val.shape}')
print(f'generation_signals_test:{generation_signals_test.shape}')

# 创建数据集
train_dataset = SignalDataset(generation_signals_train, labels_train, experimental_signal_train)
val_dataset = SignalDataset(generation_signals_val, labels_val, experimental_signal_val)
test_dataset = SignalDataset(generation_signals_test, labels_test, experimental_signal_test)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
generation_signals_train_1 = generation_signals_train.reshape(-1,M,1)
plt.style.use('default')
plt.figure(figsize=(10,6))
plt.rcParams['font.family'] = ['Times New Roman']
plt.plot(generation_signals_train_1[10],linewidth=1.5)
plt.xlabel('Sample point',fontdict={'weight': 'normal', 'size': 18})
plt.ylabel('Amplitude(V)',fontdict={'weight': 'normal', 'size': 18})
#坐标轴刻度大小设置
plt.tick_params(axis='both', which='major', labelsize=15)
plt.xlim([0,M])
plt.savefig(f'{save_path}Forward_problem/generation_signal.jpg', dpi=600, bbox_inches='tight')

In [ ]:
experimental_signal_train_1 = experimental_signal_train.reshape(-1,M,1)
plt.style.use('default')
plt.figure(figsize=(10,6))
plt.rcParams['font.family'] = ['Times New Roman']
plt.plot(experimental_signal_train_1[35],linewidth=1.5)
plt.xlabel('Sample point',fontdict={'weight': 'normal', 'size': 18})
plt.ylabel('Amplitude(V)',fontdict={'weight': 'normal', 'size': 18})
#坐标轴刻度大小设置
plt.tick_params(axis='both', which='major', labelsize=15)
plt.xlim([0,M])
plt.savefig(f'{save_path}Forward_problem/Experiment_signal.jpg', dpi=600, bbox_inches='tight')

In [9]:
def calculate_pcc(x, y):
    """
    计算两个信号的皮尔逊相关系数(PCC)。

    参数:
        x (torch.Tensor): 第一个信号，假定在 CUDA 上。
        y (torch.Tensor): 第二个信号，假定在 CUDA 上。

    返回:
        float: 皮尔逊相关系数。
    """
    
    # 计算均值
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)
    
    # 计算协方差
    covariance = torch.mean((x - mean_x) * (y - mean_y))
    
    # 计算标准差
    std_x = torch.std(x)
    std_y = torch.std(y)
    
    # 计算皮尔逊相关系数
    if std_x == 0 or std_y == 0:
        raise ValueError("标准差不能为零，这可能导致除以零的错误。")
    
    pcc = covariance / (std_x * std_y)

    pcc_1 = pcc.to(device)

    return pcc_1.item()  # 将结果转换为 Python 标量

In [10]:
# 更简单的版本（如果你只需要幅值损失）
def fft_difference(signal1, signal2, criterion=nn.MSELoss()):
    """
    计算FFT幅值差异的简化版本
    """
    
    batch_size, _, N = signal1.shape
    
    # 计算FFT
    fft1 = torch.fft.fft(signal1.squeeze(1), dim=-1)
    fft2 = torch.fft.fft(signal2.squeeze(1), dim=-1)
    
    # 取前半部分并计算归一化幅值
    amplitude1 = torch.abs(fft1[:, :N//2]) 
    amplitude2 = torch.abs(fft2[:, :N//2])
    
    # 计算损失
    fft_loss = criterion(amplitude1, amplitude2)
    
    return fft_loss


In [11]:
A = 1
# 定义训练和验证函数
def train_model(model, train_loader, val_loader, epochs, learning_rate, patience):
    """
    :param model: 模型
    :param train_loader: 训练数据加载器
    :param val_loader: 验证数据加载器
    :param epochs: 总训练轮数
    :param learning_rate: 学习率
    :param patience: 早停的耐心值，即在验证损失没有改善时等待的轮数
    :return: 训练损失、验证损失、训练PCC、验证PCC、训练FFT、验证FFT
    """
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_losses = []
    val_losses = []

    train_PCC = []
    valid_PCC = []

    train_FFT = []
    valid_FFT = []

    best_val_loss = float('inf')  # 初始化最佳验证损失为无穷大
    best_epoch = 0  # 初始化最佳轮数
    early_stopping_triggered = False  # 早停标志

    for epoch in range(epochs):
        if early_stopping_triggered:
            break  # 如果触发早停，则退出训练循环

        train_PCC_2 = 0.0
        train_FFT_2 = 0.0

        model.train()
        running_loss = 0.0
        for batch in train_loader:
            generation_signal = batch['generation_signal'].to(device)
            labels = batch['labels'].to(device)
            experimental_signal = batch['experimental_signal'].to(device)

            optimizer.zero_grad()
            experimental_signal_pred = model(generation_signal, labels)

            FFT_Amp = fft_difference(experimental_signal_pred, experimental_signal)
            train_FFT_2 += FFT_Amp.item()

            PCC_1 = calculate_pcc(experimental_signal_pred, experimental_signal)
            train_PCC_2 += PCC_1

            loss = A * criterion(experimental_signal_pred, experimental_signal)

            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_losses.append(train_loss)

        train_mean_PCC = train_PCC_2 / len(train_loader)
        train_PCC.append(train_mean_PCC)

        train_loss_FFT = train_FFT_2 / len(train_loader)
        train_FFT.append(train_loss_FFT)

        model.eval()

        running_loss = 0.0
        valid_PCC_2 = 0.0
        valid_FFT_2 = 0.0

        with torch.no_grad():
            for batch in val_loader:
                generation_signal = batch['generation_signal'].to(device)
                labels = batch['labels'].to(device)
                experimental_signal = batch['experimental_signal'].to(device)

                experimental_signal_pred = model(generation_signal, labels)

                PCC_1 = calculate_pcc(experimental_signal_pred, experimental_signal)
                valid_PCC_2 += PCC_1

                FFT_Amp = fft_difference(experimental_signal_pred, experimental_signal)
                valid_FFT_2 += FFT_Amp.item()

                loss = A * criterion(experimental_signal_pred, experimental_signal)

                running_loss += loss.item()
        val_loss = running_loss / len(val_loader)
        val_losses.append(val_loss)

        valid_mean_PCC = valid_PCC_2 / len(val_loader)
        valid_PCC.append(valid_mean_PCC)

        valid_loss_FFT = valid_FFT_2 / len(val_loader)
        valid_FFT.append(valid_loss_FFT)

        # 检查是否需要早停
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            # 保存整个模型
            torch.save(model, f'{save_path}Forward_problem/Forward_Model.pth')
            patience_counter = 0  # 重置耐心计数器
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered after {epoch + 1} epochs. Best validation loss: {best_val_loss:.4f} at epoch {best_epoch + 1}.")
                early_stopping_triggered = True

        if epoch % (epochs // 10) == 0:
            print(f"Epoch: {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}\n"
                  f"train_loss_FFT: {train_loss_FFT:.8f}, valid_loss_FFT: {valid_loss_FFT:.8f}\n"
                  f"train_mean_PCC: {train_mean_PCC:.4f}, valid_mean_PCC: {valid_mean_PCC:.4f}")

    return train_losses, val_losses, train_PCC, valid_PCC, train_FFT, valid_FFT

In [ ]:
# 训练模型
epochs = 5000
lr = 1e-4
model = ConvAutoencoder(signal_length = M, label_dim = 4, output_shape = M, hidden_dim = 128, dropout_rate = 0.2).to(device)
# 假设你已经定义了模型、训练数据加载器、验证数据加载器、损失函数等
train_losses, val_losses, train_PCC, valid_PCC, train_FFT, valid_FFT = train_model(model, train_loader, val_loader, epochs=epochs, learning_rate=lr, patience = 4000)

In [ ]:
def plot_loss_and_metrics(train_losses, val_losses, train_pccs, valid_pccs, train_ffts, valid_ffts, font='Times new roman', font_size=22, line_width=2):
    """
    绘制训练和验证损失函数图,并同时绘制PCC和FFT值。

    参数：
    train_losses (list): 训练损失值列表。
    val_losses (list): 验证损失值列表。
    train_pccs (list): 训练PCC值列表。
    valid_pccs (list): 验证PCC值列表。
    train_ffts (list): 训练FFT值列表。
    valid_ffts (list): 验证FFT值列表。
    font (str): 字体，默认为 'Arial'。
    font_size (int): 字号，默认为 12。
    line_width (float): 线条粗细，默认为 2。
    """
    # 创建主图
    plt.figure(figsize=(12, 8))
    ax1 = plt.gca()  # 获取当前主图的轴对象

    # 绘制损失
    ax1.plot(train_losses, label='Training Time-domain Loss', color='red', linewidth=line_width, marker='o')
    ax1.plot(val_losses, label='Validation Time-domain Loss', color='red', linewidth=line_width, linestyle='--', marker='s')

    # 设置主图的标签和标题
    ax1.set_xlabel('Epochs', fontname=font, fontsize=font_size)
    ax1.set_ylabel('Time-domain Loss', fontname=font, fontsize=font_size, color='red')
    ax1.tick_params(axis='both', which='major',labelcolor='red', labelsize=font_size)

    # 创建第二个Y轴
    ax2 = ax1.twinx()
    ax2.plot(train_ffts, label='Training Frequency-domain Loss', color='green', linewidth=line_width, marker='o')
    ax2.plot(valid_ffts, label='Validation Frequency-domain Loss', color='green', linewidth=line_width, linestyle='--', marker='s')
    ax2.set_ylabel('Frequency-domain Loss', fontname=font, fontsize=font_size, color='green')
    ax2.tick_params(axis='y', which='major',labelcolor='green', labelsize=font_size)

    # 创建第三个Y轴
    ax3 = ax1.twinx()
    ax3.spines['right'].set_position(('outward', 80))  # 将第三个Y轴向右移动
    ax3.plot(train_pccs, label='Training PCC', color='black', linewidth=line_width, marker='o')
    ax3.plot(valid_pccs, label='Validation PCC', color='black', linewidth=line_width, linestyle='--', marker='s')
    ax3.set_ylabel('PCC', fontname=font, fontsize=font_size, color='black')
    ax3.tick_params(axis='y', which='major',labelcolor='black', labelsize=font_size)

    # 将图例放在一起
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    lines3, labels3 = ax3.get_legend_handles_labels()
    ax1.legend(lines + lines2 + lines3, labels + labels2 + labels3, loc=(0.1,0.2), fontsize=font_size)
    
    # 保存图像
    plt.savefig(f'{save_path}Forward_problem/Forward_Model_Loss.jpg', dpi=300, bbox_inches='tight')
    plt.show()

# 调用函数，传入你的数据
plot_loss_and_metrics(train_losses, val_losses, train_PCC, valid_PCC, train_FFT, valid_FFT)

In [ ]:
def plot_loss_and_metrics(train_losses, val_losses, train_pccs, valid_pccs, train_ffts, valid_ffts, font='Times new roman', font_size=22, line_width=2):
    """
    绘制训练和验证损失函数图,并同时绘制PCC和FFT值。

    参数：
    train_losses (list): 训练损失值列表。
    val_losses (list): 验证损失值列表。
    train_pccs (list): 训练PCC值列表。
    valid_pccs (list): 验证PCC值列表。
    train_ffts (list): 训练FFT值列表。
    valid_ffts (list): 验证FFT值列表。
    font (str): 字体，默认为 'Arial'。
    font_size (int): 字号，默认为 12。
    line_width (float): 线条粗细，默认为 2。
    """
    # 创建主图
    plt.figure(figsize=(12, 8))
    ax1 = plt.gca()  # 获取当前主图的轴对象

    # 绘制损失
    ax1.plot(train_losses, label='Training Time-domain Loss', color='red', linewidth=line_width, marker='o')
    ax1.plot(val_losses, label='Validation Time-domain Loss', color='red', linewidth=line_width, linestyle='--', marker='s')
  
    y_min = min(min(train_losses[int(len(train_losses)*0.8):len(train_losses)]),min(val_losses[int(len(train_losses)*0.8):len(train_losses)]))
    y_max = max(max(train_losses[int(len(train_losses)*0.8):len(train_losses)]),max(val_losses[int(len(train_losses)*0.8):len(train_losses)]))
    # 设置主图的标签和标题
    ax1.set_xlabel('Epochs', fontname=font, fontsize=font_size)
    ax1.set_ylabel('Time-domain Loss', fontname=font, fontsize=font_size, color='red')
    ax1.tick_params(axis='both', which='major',labelcolor='red', labelsize=font_size)
    plt.xlim(int(len(train_losses)*0.8),len(train_losses))
    plt.ylim(y_min,y_max)

    # 保存图像
    plt.savefig(f'{save_path}Forward_problem/Model_Loss_Enlargement_Time.jpg', dpi=300, bbox_inches='tight')
    plt.show()

# 调用函数，传入你的数据
plot_loss_and_metrics(train_losses, val_losses, train_PCC, valid_PCC, train_FFT, valid_FFT)

In [ ]:
def plot_loss_and_metrics(train_losses, val_losses, train_pccs, valid_pccs, train_ffts, valid_ffts, font='Times new roman', font_size=22, line_width=2):
    """
    绘制训练和验证损失函数图,并同时绘制PCC和FFT值。

    参数：
    train_losses (list): 训练损失值列表。
    val_losses (list): 验证损失值列表。
    train_pccs (list): 训练PCC值列表。
    valid_pccs (list): 验证PCC值列表。
    train_ffts (list): 训练FFT值列表。
    valid_ffts (list): 验证FFT值列表。
    font (str): 字体，默认为 'Arial'。
    font_size (int): 字号，默认为 12。
    line_width (float): 线条粗细，默认为 2。
    """
    # 创建主图
    plt.figure(figsize=(12, 8))
    ax1 = plt.gca()  # 获取当前主图的轴对象
    ax1.plot(train_ffts, label='Training Frequency-domain Loss', color='green', linewidth=line_width, marker='o')
    ax1.plot(valid_ffts, label='Validation Frequency-domain Loss', color='green', linewidth=line_width, linestyle='--', marker='s')
    
    y_min = min(min(train_ffts[int(len(train_ffts)*0.8):len(train_ffts)]),min(valid_ffts[int(len(valid_ffts)*0.8):len(valid_ffts)]))
    y_max = max(max(train_ffts[int(len(train_ffts)*0.8):len(train_ffts)]),max(valid_ffts[int(len(valid_ffts)*0.8):len(valid_ffts)]))
    
    ax1.set_ylabel('Frequency-domain Loss', fontname=font, fontsize=font_size, color='green')
    ax1.tick_params(axis='x', which='major',labelcolor='green', labelsize=font_size)
    ax1.tick_params(axis='y', which='major',labelcolor='green', labelsize=font_size)
    plt.xlim(int(len(train_ffts)*0.8),len(train_ffts))
    plt.ylim(y_min,y_max)

    # 保存图像
    plt.savefig(f'{save_path}Forward_problem/Model_Loss_Enlargement_Frequency.jpg', dpi=300, bbox_inches='tight')
    plt.show()

# 调用函数，传入你的数据
plot_loss_and_metrics(train_losses, val_losses, train_PCC, valid_PCC, train_FFT, valid_FFT)

In [ ]:
def plot_loss_and_metrics(train_losses, val_losses, train_pccs, valid_pccs, train_ffts, valid_ffts, font='Times new roman', font_size=22, line_width=2):
    """
    绘制训练和验证损失函数图,并同时绘制PCC和FFT值。

    参数：
    train_losses (list): 训练损失值列表。
    val_losses (list): 验证损失值列表。
    train_pccs (list): 训练PCC值列表。
    valid_pccs (list): 验证PCC值列表。
    train_ffts (list): 训练FFT值列表。
    valid_ffts (list): 验证FFT值列表。
    font (str): 字体，默认为 'Arial'。
    font_size (int): 字号，默认为 12。
    line_width (float): 线条粗细，默认为 2。
    """
    # 创建主图
    plt.figure(figsize=(12, 8))
    ax1 = plt.gca()  # 获取当前主图的轴对象
    ax1.plot(train_pccs, label='Training PCC', color='black', linewidth=line_width, marker='o')
    ax1.plot(valid_pccs, label='Validation PCC', color='black', linewidth=line_width, linestyle='--', marker='s')

    y_min = min(min(train_pccs[int(len(train_pccs)*0.8):len(train_pccs)]),min(valid_pccs[int(len(valid_pccs)*0.8):len(valid_pccs)]))
    y_max = max(max(train_pccs[int(len(train_pccs)*0.8):len(train_pccs)]),max(valid_pccs[int(len(valid_pccs)*0.8):len(valid_pccs)]))

    ax1.set_ylabel('PCC', fontname=font, fontsize=font_size, color='black')
    ax1.tick_params(axis='x', which='major',labelcolor='black', labelsize=font_size)
    ax1.tick_params(axis='y', which='major',labelcolor='black', labelsize=font_size)

    plt.xlim(int(len(train_pccs)*0.8),len(train_pccs))
    plt.ylim(y_min,y_max)

    # 保存图像
    plt.savefig(f'{save_path}Forward_problem/Model_Loss_Enlargement_PCC.jpg', dpi=300, bbox_inches='tight')
    plt.show()

# 调用函数，传入你的数据
plot_loss_and_metrics(train_losses, val_losses, train_PCC, valid_PCC, train_FFT, valid_FFT)